# [ISLR] Chapter 03 - Linear Regression

* ISLR 책을 공부하며 가벼운 내용 소개, 원래 알고 있던 내용을 함께 정리합니다.
* ISLR 책에 대한 원서는 [여기](http://www-bcf.usc.edu/~gareth/ISL/)에서 확인할 수 있습니다.
* ISLR 책에 대한 참고 블로그는 [Go`s Blog](https://godongyoung.github.io/category/ML.html)를 참고했습니다.
* 코드 구현은 [밑바닥부터 시작하는 데이터 과학](https://book.naver.com/bookdb/book_detail.nhn?bid=10652749) 이 책을 참고했습니다.
* 오타나 틀린 내용, 토의할 내용은 언제나 이슈로 등록해주세요.
---

## Contents
    3.1 Simple Linear Regression
        3.1.1 Estimating the Coefficients
        3.1.2 Assessing the Accuracy of the Coefficient Estimates
        3.1.3 Assessing the Accuracy of the Model
    
    3.2 Multiple Linear Regression
        3.2.1 Estimating the Regression Coefficients
        3.2.2 Some Important Questions
    
    3.3 Other Considerations in the Regression Model
        3.3.1 Qualitative Predictors
        3.3.2 Extensions of the Linear Model
        3.3.3 Potential Problems
        
    3.4 The Marketing Plan
    
    3.4 Comparison of Linear Regression with K-Nearest Neighbors
---